In [1]:
import pandas as pd

# Load the datasets
no_sp_df = pd.read_csv('../log/tmplog_finalTest3.csv')
sp_df = pd.read_csv('../log/tmplog_finalTest3_SP.csv')

# Display the first few rows of each DataFrame to understand the structure
no_sp_df.head(), sp_df.head()

(                       flow id:  first send:  last send:  last arrive:  \
 0  11:0:11 11:0:41 10099 100 17     2.004457  235.999032    236.006409   
 1  11:0:11 11:0:41 10097 100 17     2.004407  142.265974    142.285869   
 2  11:0:11 11:0:41 10096 100 17     2.004382  353.654458    353.658402   
 3  11:0:11 11:0:41 10093 100 17     2.004256  150.274747    150.292520   
 4  11:0:11 11:0:41 10092 100 17     2.004231  140.723312    140.740632   
 
    send num:  receive num:  send size:  receive size:        FCT:  origin_pg:  \
 0     925003        925147    10364472       10364472  234.001952           3   
 1      83824         83859     2542913        2542913  140.281462           3   
 2    5465426       5465638    30511896       30511896  351.654019           3   
 3     141217        141268     3335902        3335902  148.288264           3   
 4      58070         58091     1198573        1198573  138.736401           3   
 
    transfer delay:  
 0                0  
 1        

In [2]:
# Merge the datasets on 'flow id:' column
merged_df = pd.merge(no_sp_df, sp_df, on='flow id:', suffixes=('_no_sp', '_sp'))

# Calculate FCT change and percentage change
merged_df['FCT Change'] = merged_df['FCT:_sp'] - merged_df['FCT:_no_sp']
merged_df['FCT % Change'] = (merged_df['FCT Change'] / merged_df['FCT:_no_sp']) * 100

# Display the new dataframe with changes
merged_df[['flow id:', 'FCT:_no_sp', 'FCT:_sp', 'FCT Change', 'FCT % Change']].head()


,flow id:,FCT:_no_sp,FCT:_sp,FCT Change,FCT % Change
0,11:0:11 11:0:41 10099 100 17,234.001952,536.151855,302.149903,129.122813
1,11:0:11 11:0:41 10097 100 17,140.281462,228.399524,88.118062,62.815186
2,11:0:11 11:0:41 10096 100 17,351.654019,785.964024,434.310005,123.504917
3,11:0:11 11:0:41 10093 100 17,148.288264,264.735879,116.447615,78.527870
4,11:0:11 11:0:41 10092 100 17,138.736401,182.226737,43.490336,31.347459


In [3]:
merged_df

,flow id:,first send:_no_sp,last send:_no_sp,last arrive:_no_sp,send num:_no_sp,receive num:_no_sp,send size:_no_sp,receive size:_no_sp,FCT:_no_sp,origin_pg:_no_sp,...,last arrive:_sp,send num:_sp,receive num:_sp,send size:_sp,receive size:_sp,FCT:_sp,origin_pg:_sp,transfer delay:_sp,FCT Change,FCT % Change
0,11:0:11 11:0:41 10099 100 17,2.004457,235.999032,236.006409,925003,925147,10364472,10364472,234.001952,3,...,538.156312,2135994,2136461,10364472,10364472,536.151855,3,0,302.149903,129.122813
1,11:0:11 11:0:41 10097 100 17,2.004407,142.265974,142.285869,83824,83859,2542913,2542913,140.281462,3,...,230.403930,176164,176310,2542913,2542913,228.399524,3,0,88.118062,62.815186
2,11:0:11 11:0:41 10096 100 17,2.004382,353.654458,353.658402,5465426,5465638,30511896,30511896,351.654019,3,...,787.968406,12258485,12259144,30511896,30511896,785.964024,3,0,434.310005,123.504917
3,11:0:11 11:0:41 10093 100 17,2.004256,150.274747,150.292520,141217,141268,3335902,3335902,148.288264,3,...,266.740135,301627,301815,3335902,3335902,264.735879,3,0,116.447615,78.527870
4,11:0:11 11:0:41 10092 100 17,2.004231,140.723312,140.740632,58070,58091,1198573,1198573,138.736401,3,...,184.230968,78139,78212,1198573,1198573,182.226737,3,0,43.490336,31.347459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,11:0:31 11:0:41 10053 100 17,2.001633,140.064685,140.080228,275157,275246,338595,338595,138.078595,2,...,134.810925,266022,266024,338595,338595,132.809292,2,0,-5.269303,-3.816162
296,11:0:31 11:0:41 10054 100 17,2.001676,217.708906,217.717421,894433,894573,965094,965094,215.715745,2,...,155.761136,622890,622892,965094,965094,153.759460,2,0,-61.956285,-28.721263
297,11:0:31 11:0:41 10008 100 17,2.000250,127.364416,127.382567,2211,2211,2362,2362,125.382317,1,...,127.389510,2211,2211,2362,2362,125.389260,1,0,0.006944,0.005538
298,11:0:31 11:0:41 10058 100 17,2.002563,113.617180,113.633958,119663,119700,2611316,2611316,111.631395,3,...,191.659597,195951,196102,2611316,2611316,189.657034,3,0,78.025639,69.895785


In [4]:
# Group by origin priority group and calculate the required statistics
group_stats = merged_df.groupby('origin_pg:_no_sp').agg(
    FCT_Total_No_SP=pd.NamedAgg(column='FCT:_no_sp', aggfunc='sum'),
    FCT_Total_SP=pd.NamedAgg(column='FCT:_sp', aggfunc='sum'),
    Total_FCT_Change=pd.NamedAgg(column='FCT Change', aggfunc='sum'),
    Mean_FCT_Change=pd.NamedAgg(column='FCT % Change', aggfunc='mean'),
    Mean_FCT_Per_Change=pd.NamedAgg(column='FCT Change', aggfunc='mean'),
    Count_FCT_Decrease=pd.NamedAgg(column='FCT Change', aggfunc=lambda x: (x <= 0.0).sum()),
    Total_Flows=pd.NamedAgg(column='FCT Change', aggfunc='size')
)

# Calculate percentage of flows with decreased FCT
group_stats['Percentage_FCT_Decrease'] = (group_stats['Count_FCT_Decrease'] / group_stats['Total_Flows']) * 100

# Display the summarized statistics
group_stats.reset_index()


,origin_pg:_no_sp,FCT_Total_No_SP,FCT_Total_SP,Total_FCT_Change,Mean_FCT_Change,Mean_FCT_Per_Change,Count_FCT_Decrease,Total_Flows,Percentage_FCT_Decrease
0,1,15489.357790,15483.263805,-6.093985,-0.046837,-0.045141,91,135,67.407407
1,2,20522.411107,19813.773648,-708.637459,-10.610263,-23.621249,19,30,63.333333
2,3,61284.080622,98598.216539,37314.135917,86.139255,276.401007,4,135,2.962963


In [5]:
merged_df.to_csv("./result.csv")

In [6]:
#0 	1 	15470.217402 	15470.217336 	-0.000066 	-4.888741e-07 	1 	135 	0.740741

In [7]:
# in 40Mpbs some high pg flows' fct has increase as follow: 
    # 131,11:0:21 11:0:41 10020 100 17,2.00055418,125.34508296,125.364080097,2083,2086,123.363525917,1,0,2.00055418,125.366682448,125.366845447,2086,2086,123.366291267,1,0,0.0027653499999900077,0.002241626914790485
        # use high_60 replace high_61
    # 156,11:0:21 11:0:41 10004 100 17,2.000111196,113.565199616,113.58213076,1627,1629,111.582019564,1,0,2.000111196,113.610388046,113.610699046,1629,1629,111.61058785,1,0,0.02856828599999517,0.02560294760000224
        # use high_12 replace high_13
    # 183,11:0:21 11:0:41 10040 100 17,2.001132561,120.484628618,120.506937341,1108,1111,118.50580478,1,0,2.001132561,120.556739024,120.557038222,1113,1114,118.555905661,1,0,0.05010088099999166,0.04227715350568809
        # use high_120 replace high_121
    # 180,11:0:21 11:0:41 10029 100 17,2.000844572,121.428252899,121.447419428,1951,1951,119.446574856,1,0,2.000844572,121.44740644,121.447481839,1951,1951,119.446637267,1,0,6.241100000181632e-05,5.225013783530965e-05
        # use high_87 replace high_88   
    # 256,11:0:31 11:0:41 10022 100 17,2.000626577,123.632155519,123.651544052,2069,2082,121.650917475,1,0,2.000626577,123.653212468,123.653460069,2069,2069,121.652833492,1,0,0.001916016999999215,0.0015750123712737045
        # use high_67 replace high_68       
    # 50,11:0:11 11:0:41 10020 100 17,2.00054058,124.004466398,124.021815529,2071,2075,122.021274949,1,0,2.00054058,124.023811895,124.023887895,2071,2071,122.023347315,1,0,0.0020723659999930533,0.0016983644867333333
        # use high_59 replace high_60
        # use high_60 replace high_61
